In [1]:
# This notebook uses a nomogram available on the web to create an additional feature for patients
# The nomogram is available at https://nomograms.mskcc.org/Breast/BreastSLNodeMetastasisPage.aspx
# The filling and submitting of the webform is automated using the 'pyautogui' module
from pyautogui import * 
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsd
import pyperclip

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

nomogram_url = "https://nomograms.mskcc.org/Breast/BreastSLNodeMetastasisPage.aspx"
raw_data_path = "/Users/yifu/PycharmProjects/Radiotherapy-Prediction/data/input/AllTranTrainVal.csv"

In [2]:
# Load raw data
df = pd.read_csv(raw_data_path)
df.head()

,record_id,dob,men_status,fmhx,gensus___1,gensus___2,gensus___3,gensus___4,gensus___5,gensus___6,gensus___7,systhe___1,systhe___2,systhe___3,systhe___4,systhe___5,tumor_laterality,height_cm,weight_kg,bra_cup_size,bra_cup_size_measure,palpability,axillary_lymph_node_palpab,dximg___1,dximg___2,dximg___3,dximg_date,bi_rads_score,img_size,foci,tumor_stge,abnormal_lymph,lymph_node_max_size_mm,tumor_size_mm,microcalcifications,extent_of_calcification_ma,prominent_axillary_lymph,lymph_node_max_size_mm0,backgroun_enhancement,max_enhancement_measurment,axillary_lymphadenopathy,internal_mammary_lymphaden,high_grade_fdg_foci_presen,size_of_the_largest_foci_c,axillary_lymphadenopathy_p,axillary_lymph_node_max_si,int_mammary_lymphade_pet,internal_mammary_lymph_nod,pre_op_biop_date,pre_op_biopsy,tumor_location,tumor_location_trans,his_subtype___1,his_subtype___2,his_subtype___3,his_subtype___4,his_subtype___5,his_subtype___6,specify_histology_if_non_o,tumor_grade,margin_status,closest_margin,closest_margin_trans,distance_from_closest_marg,lymphovascular_invasion0,er_status,pr_status,her_status,imaging_and_biopsy_concord,axillary_lymph_node_core_b,metastatic_carcinoma_on_ax,surgical_indication1_primary_treatment___1,surgical_indication1_primary_treatment___2,surgical_indication1_primary_treatment___3,surgical_indication1_primary_treatment___4,surgical_indication1_primary_treatment___5,laterality,surgery_date,breast_procedure,axillary_surgery___1,axillary_surgery___2,axillary_surgery___3,lymph_nodes,sln,number_of_positive_sln,ax_nodes,mastectomy_weight_g,tumor_size,tumor_loc,his_type___1,his_type___2,his_type___3,his_type___4,his_type___5,his_type___6,his_type___7,his_type___8,specify_histology_if_other,tu_grade,tumor_focality,num_foci,lymphovascular_invasion,in_situ_component_present,in_situ_component_type___1,in_situ_component_type___2,in_situ_component_size_mm,in_situ_component_grade,mar_status,clos_margin___1,clos_margin___2,clos_margin___3,clos_margin___4,clos_margin___5,clos_margin___6,clos_margin___7,dis_closest_margin,metastasis,ln_w_micrometastasis,ln_w_macrometastasis,size_of_largest_nodal_meta,extranodal_extension,extent_of_extranodal_exten,did_the_patient_receive_pm,did_the_patient_reject_pmr,data_collection_fields_complete
0,ANN001,10/1/60,2.0,0.0,0,0,0,0,0,1,0,1,0,0,0,0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1,1,0,1/31/18,NaN,49.0,NaN,NaN,1.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.9,1.0,31.0,0.0,NaN,NaN,1.0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,1.0,0.0,1,0,0,0,0,3.0,9/28/18,3.0,0,0,1,16.0,0.0,0.0,0.0,NaN,2.0,NaN,1,0,0,0,0,0,0,0,NaN,3.0,NaN,NaN,2.0,NaN,0,0,NaN,NaN,2.0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2
1,ANN002,1/1/66,2.0,0.0,0,0,0,0,0,1,0,0,0,0,1,0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,1,1,0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,retroarolar,NaN,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,1.0,0.0,NaN,1,0,0,0,0,1.0,10/11/18,3.0,0,1,0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,2.0,1.0,1,0,15,2.0,2.0,1,0,0,0,0,0,0,20,2.0,NaN,NaN,NaN,2.0,NaN,0.0,NaN,2
2,ANN003,6/1/62,2.0,0.0,0,0,0,0,0,1,0,0,0,0,1,0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,0,1,0,3/22/18,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,7/4/18,2.0,NaN,NaN,1,0,0,0,0,0,NaN,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,2.0,1.0,0.0,NaN,0,0,1,0,0,1.0,NaN,3.0,0,1,0,3.0,3.0,0.0,0.0,NaN,30.0,6 o'clock,1,0,0,0,0,0,0,0,NaN,1.0,NaN,NaN,2.0,NaN,0,0,NaN,NaN,1.0,0,0,0,0,0,0,0,1,2.0,NaN,NaN,NaN,2.0,NaN,0.0,NaN,2
3,ANN004,11/1/60,2.0,1.0,1,0,0,0,0,0,0,1,0,0,0,0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1,0,0,10/9/18,NaN,14.0,NaN,NaN,2.0,NaN,14.0,0.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/16/18,1.0,2 o'clock,2.0,1,0,0,0,0,0,NaN,2.0,NaN,NaN,NaN,NaN,0.0,2.0,2.0,2.0,1.0,0.0,0.0,1,0,0,0,0,3.0,12/5/18,3.0,0,1,0,1.0,3.0,NaN,0.0,NaN,11.0,NaN,1,0,0,0,0,0,0,0,NaN,3.0,1.0,NaN,2.0,1.0,1,0,NaN,3.0,2.0,0,1,0,0,0,0,0,3,2.0,NaN,NaN,NaN,2.0,NaN,0.0,NaN,2
4,ANN005,10/1/59,2.0,0.0,

In [3]:
"""
Create a subset of columns required by nomogram

Nomogram website: https://nomograms.mskcc.org/Breast/BreastSLNodeMetastasisPage.aspx

The columns are:
Current Age (20 to 91 yrs)
    Enter current age. Must be between 20 and 91. 
Breast Tumor Size  (0.1 to 11.0 cm)
    Size of the primary tumor (as measured either in imaging study or pathological exam), in centimeters.
Special Type? (boolean)
    Check box if tumor has been pathologically defined as pure tubular, pure colloid (mucinous), or typical medullary carcinomas
    on the pathology report. Other histologies such as atypical medullary carcinoma or carcinoma with ductal and lobular features should 
    be classified as ductal -- see Tumor Type and Grade section below for more details.
Tumor is confined to UIQ? (boolean)
    Check box if tumor is confined within the upper inner quadrant (UIQ) of the breast.
Lymphatic or Vascular Structure Involvement (boolean)
    Check box if one or more tumor cells found in the blood or lymphatic vessels.
Multifocality? (boolean)
    Check box if breast cancer has cancer cells seperated from the main tumor mass.
Tumor Type and Grade (Ductal I, Ductal II, Ductal III, Lobular)
    Indicate if tumor type is ductal or lobular, as noted in the pathology report.
    If ductal, indicate the nuclear grade -- 
        I: slight or no variation in the size and shape of the nucleus;
        II: moderate variation in the size and shape of the nucleus;
        III: marked variation in the size and shape of the nucleus.
Estrogen-Receptor Status (Negative, Positive, Unknown)
    Select NEGATIVE if estrogen receptors stain positive in <10% of cells;
    Select POSITIVE if estrogen receptors stain positive in ≥10% of cells.
Progesterone-Receptor Status
    Select NEGATIVE if progesterone receptors stain positive in <10% of cells;
    select POSITIVE if progesterone receptors stain positive in ≥10% of cells.
"""

from typing import Dict, List

class FieldType:
    """
    Enum for field type
    """
    CHECKBOX = "CHECKBOX"
    DROPDOWN = "DROPDOWN"
    TEXTAREA = "TEXTAREA"
    RESULT = "RESULT"
    CALCULATE = "CALCULATE"
    CLEAR = "CLEAR"
    

class NomogramField:
    
    def __init__(self, web_field_name: str, web_field_type: FieldType, df_column_name: str, dropdown_options: List[str] = []):
        """Object for holding and structuring the information to fill the webform automatically

        Args:
            web_field_name (str): The name as it appears on the nomogram website
            web_field_type (FieldType): The type of the webform field
            df_column_name (str): The name of column in the local dataset
        """
        self.web_field_name = web_field_name
        self.web_field_type = web_field_type
        self.df_column_name = df_column_name
        self.webform_x_coord = None
        self.webform_y_coord = None
        self.dropdown_coords = None
        self.dropdown_options = dropdown_options
    
    def update_webform_coordinates(self):
        """
        Update the coordinates of the webform field using the current mouse location
        """
        field_type = self.web_field_type
        alert(
            f"Calibrating the screen coordinates of '{self.web_field_name}' on webform...\n"
            "\n\nINSTRUCTIONS:\n\n"
            "Place your mouse where the field can be filled, and press Enter without moving the mouse.\n" \
            "The very tip of the mouse pointer should be INSIDE the checkbox, text area, or drop down menu."
            "The location/scrolling of the webform should not be changed during the entire calibration process for all fields.",
            title=f"Calibrating Screen Coordinates",
            button="Press Enter while holding the mouse in place to continue"
            )
        self.webform_x_coord, self.webform_y_coord = position()
        alert(
            f"Coordinates of '{self.web_field_name}' updated: {self.webform_x_coord}, {self.webform_y_coord}",
            title="Coordinates Updated", 
            button="Press Enter to continue"
        )
        if field_type == FieldType.DROPDOWN:
            # alert(
            #     f"'{self.web_field_name}' is a dropdown menu, calibration is needed for each option available.\n"
            #     f"Please click the blank dropdown menu to open it.\n"
            #     "Do not click/press anything else. Please continue hold the mouse in place and wait until 10 seconds have passed.",
            #     button="(automatically continuing in 10 seconds)",
            #     title="Further Calibration Needed"
            # )
            dropdown_coords = {}
            for option in self.dropdown_options:
                alert(
                    f"'{self.web_field_name}' is a dropdown menu, calibration is needed for each option available.\n"
                    "Please make sure the dropdown menu isn't chosen (completely blank)\n"
                    "Then, click the blank dropdown menu to open it.\n"
                    f"Then, hover the tip of the mouse pointer on the dropdown option for '{option}'\n"
                    "Then, mark the location of the tip of the mouse pointer on your screen using your finger or pen."
                    "Then, come back to this message box, click this box to make window active, and"
                    "hover your mouse back to the marked location. Lastly, press Enter to continue.\n",
                    button="Press Enter to continue",
                    title=f"Calibrating for {option}"
                )
                option_x_coord, option_y_coord = position()
                alert(
                    f"Coordinates of '{self.web_field_name}' = '{option}' updated: {option_x_coord}, {option_y_coord}",
                    title="Coordinates Updated", 
                    button="Press Enter to continue"
                )
                dropdown_coords[option] = (option_x_coord, option_y_coord)
            self.dropdown_coords = dropdown_coords
        elif field_type == FieldType.RESULT:
            result_coords = {}
            alert(
                "Move the cursor to the TOP-LEFT corner of the grey box where the nomogram will output the probabilities "
                "This script will drag across the area to select the resulting probability to be copied.",
                title="Calibrating for Dragging and Copying Result",
                button="Press Enter to continue"
            )
            result_coords["top_left"] = position()
            alert(
                "Move the cursor to the BOTTOM-RIGHT corner of the grey box where the nomogram will output the probabilities "
                "This script will drag across the area to select the resulting probability to be copied.",
                title="Calibrating for Dragging and Copying Result",
                button="Press Enter to continue"
            )
            result_coords["bottom_right"] = position()
            self.result_coords = result_coords

    def fill_webform(self, web_field_name: str, row: pd.Series):
        """Given a row in dataframe, fill the field in webform

        Args:
            web_field_name (str): The name as it appears on the nomogram website
            row (pd.Series): The row in the dataframe
        Raises:
            ValueError: If the field is not supported, or if value is invalid
        """
        field_type = self.web_field_type
        df_column = self.df_column_name
        webform_x_coord = self.webform_x_coord
        webform_y_coord = self.webform_y_coord
        dropdown_coords = self.dropdown_coords
        value = row[df_column]
        if field_type == FieldType.CHECKBOX:
            if value == "True" and ...:
                raise NotImplementedError
                # Need to toggle the checkbox if it's previously unchecked
            elif value == "False":
                raise NotImplementedError
                # click(webform_x_coord, webform_y_coord)
        elif field_type == FieldType.DROPDOWN:
            doubleClick(webform_x_coord, webform_y_coord)
            doubleClick(webform_x_coord, webform_y_coord)
            time.sleep(0.1)
            x, y = self.get_dropdown_option_coords(value)
            click(x, y)
        elif field_type == FieldType.TEXTAREA:
            doubleClick(webform_x_coord, webform_y_coord)
            time.sleep(0.1)
            doubleClick(webform_x_coord, webform_y_coord)
            time.sleep(0.1)
            typewrite(value)
            time.sleep(0.1)
            press("enter")
        else:
            raise ValueError("Unsupported field type")
        
    def get_dropdown_option_coords(self, value: str):
        """Given a value, return the coordinates of the dropdown option

        Args:
            value (str): The value of the dropdown option
        Raises:
            ValueError: If the value is not supported
        """
        if value not in self.dropdown_options:
            raise ValueError(f"Unsupported dropdown option {value}.")
        else:
            return self.dropdown_coords[value]

    def fill_value(self, value):
        field_type = self.web_field_type
        webform_x_coord, webform_y_coord = self.webform_x_coord, self.webform_y_coord
        if field_type == FieldType.CLEAR:
            clear_field = nomogram_fields["Clear"]
            click(clear_field.webform_x_coord, clear_field.webform_y_coord)
            time.sleep(0.5)
            click(clear_field.webform_x_coord, clear_field.webform_y_coord)
            time.sleep(5)
        elif field_type == FieldType.TEXTAREA:
            doubleClick(webform_x_coord, webform_y_coord, interval=0.2)
            time.sleep(0.1)
            typewrite(value)
        elif field_type == FieldType.CHECKBOX:
            if value == "Yes":
                click(webform_x_coord, webform_y_coord)
        elif field_type == FieldType.DROPDOWN:
            click(webform_x_coord, webform_y_coord)
            time.sleep(0.1)
            option_x_coord, option_y_coord = self.get_dropdown_option_coords(value)
            time.sleep(0.1)
            click(option_x_coord, option_y_coord)
        elif field_type == FieldType.CALCULATE:
            click(webform_x_coord, webform_y_coord)
            time.sleep(1)
        elif field_type == FieldType.RESULT:
            time.sleep(1)
            top_left_coords = self.result_coords["top_left"]
            bottom_right_coords = self.result_coords["bottom_right"]
            moveTo(top_left_coords.x, top_left_coords.y)
            time.sleep(0.1)
            dragTo(bottom_right_coords.x, bottom_right_coords.y, duration=0.2, button="left")
            if os_type == "Windows":
                hotkey("ctrl", "c")
            elif os_type == "Mac":
                hotkey("command", "c")
            time.sleep(0.5)
            result = pyperclip.paste()
            if "%" in result:
                result = result.replace("%", "")
                result = round(float(result) / 100, 3)
                pyperclip.copy(result)
            print(f"The resulting probability is {result}")
        else:
            raise Exception(f"Unknown field type: {field_type}")
        # Press tab to clear auto-completes
        time.sleep(0.1)
        press("tab")

In [4]:
from collections import OrderedDict


nomogram_fields = OrderedDict({
    "Clear": NomogramField(
        web_field_name="Clear",
        web_field_type=FieldType.CLEAR,
        df_column_name=None
    ),
    "Current Age": NomogramField(
        web_field_name="Current Age",
        web_field_type=FieldType.TEXTAREA,
        df_column_name="nomogram_age"
    ),
    "Breast Tumor Size": NomogramField(
        web_field_name="Breast Tumor Size",
        web_field_type=FieldType.TEXTAREA,
        df_column_name="nomogram_max_tumor_size_composite"
    ),
    "Special Type?": NomogramField(
        web_field_name="Special Type?",
        web_field_type=FieldType.CHECKBOX,
        df_column_name="nomogram_is_special_type"
    ),
    "Tumor is confined...": NomogramField(
        web_field_name="Tumor is confined to UIQ?",
        web_field_type=FieldType.CHECKBOX,
        df_column_name="nomogram_tumor_confined_uiq"
    ),
    "Lymphatic or Vascular...": NomogramField(
        web_field_name="Lymphatic or Vascular Structure Involvement (LymphovascularInvasion)",
        web_field_type=FieldType.CHECKBOX,
        df_column_name="nomogram_is_lymphovascular_invasive"  # TODO whether to use POST LVI?
    ),
    "Multifocality?": NomogramField(
        web_field_name="Multifocality?",
        web_field_type=FieldType.CHECKBOX,
        df_column_name="nomogram_is_multifocal"
    ),
    "Tumor Type and Grade": NomogramField(
        web_field_name="Tumor Type and Grade",
        web_field_type=FieldType.DROPDOWN,
        df_column_name="nomogram_tumor_type_and_grade",
        dropdown_options=["Ductal, I", "Ductal, II", "Ductal, III", "Lobular"]
    ),
    "Estrogen-Receptor Status": NomogramField(
        web_field_name="Estrogen-Receptor Status",
        web_field_type=FieldType.DROPDOWN,
        df_column_name="nomogram_er_status",
        dropdown_options=["Negative", "Positive", "Unknown"]
    ),
    "Progesterone-Receptor Status": NomogramField(
        web_field_name="Progesterone-Receptor Status",
        web_field_type=FieldType.DROPDOWN,
        df_column_name="nomogram_pr_status",
        dropdown_options=["Negative", "Positive", "Unknown"]
    ),
    "Calculate": NomogramField(
        web_field_name="Calculate",
        web_field_type=FieldType.CALCULATE,
        df_column_name=None
    ),
    "Probability of SLN+": NomogramField(
        web_field_name="Probability of Spread to Sentinel Lymph Nodes (the area where the probability "
        "as a percentage will appear. Initially, it is an empty grey box, inside another grey box titled "
        "'Your Results' and to the right to the blue text 'Probability of Spread to Sentinel Lymph Nodes'.)",
        web_field_type=FieldType.RESULT,
        df_column_name=None
    )
})


def calibrate_coordinates(nomogram_fields):
    """Given a dictionary of NomogramField objects, calibrate the coordinates of each field

    Args:
        nomogram_fields (Dict[str, NomogramField]): The dictionary of NomogramField objects
    """
    # Get the user's OS whether it's Windows or Mac
    global os_type
    if sys.platform == "win32":
        os_type = "Windows"
    elif sys.platform == "darwin":
        os_type = "Mac"
    else:
        raise ValueError("Unsupported OS")
    alert(f"The current OS is: {os_type}", title="OS Detected", button="Press Enter to continue")
    time.sleep(0.5)
    alert(
        "\n\nPlease have the webform on the right half of the screen, and the Excel spreadsheet on the left half.\n"
        "The webform should be scrolled to the *top-left* of the page.\n"
        "And the webform should be zoomed-out sufficiently so that all the fields (and also the probability in Your Results on the right) are visible without scrolling.\n"
        "Please do not move the browser or scroll the webform during the entire calibration process for all fields.\n\n"
        "When you are ready, click OK or press Enter to continue.",
        title="Starting Calibration...",
        button='OK (or press Enter)'
        )
    # For each field, update the coordinates of the webform field using the current mouse location
    for field in nomogram_fields.values():
        field.update_webform_coordinates()

    confirm("Calibration is complete.")
    time.sleep(1)
    # Print all the fields and their coordinates
    print("--- the coordinates of the webform fields are ---")
    for field in nomogram_fields.values():
        print(f"{field.web_field_name}: {field.webform_x_coord}, {field.webform_y_coord}")
    time.sleep(1)
    

calibrate_coordinates(nomogram_fields)

--- the coordinates of the webform fields are ---
Clear: 936, 895
Current Age: 1034, 312
Breast Tumor Size: 1042, 350
Special Type?: 1024, 389
Tumor is confined to UIQ?: 1025, 492
Lymphatic or Vascular Structure Involvement (LymphovascularInvasion): 1024, 538
Multifocality?: 1026, 597
Tumor Type and Grade: 1032, 647
Estrogen-Receptor Status: 1041, 736
Progesterone-Receptor Status: 1045, 798
Calculate: 1087, 891
Probability of Spread to Sentinel Lymph Nodes (the area where the probability as a percentage will appear. Initially, it is an empty grey box, inside another grey box titled 'Your Results' and to the right to the blue text 'Probability of Spread to Sentinel Lymph Nodes'.): 1436, 324


In [5]:
# Clear webform

def test_fill(nomogram_fields):
    global os_type

    clear_field = nomogram_fields["Clear"]
    click(clear_field.webform_x_coord, clear_field.webform_y_coord)
    time.sleep(0.5)
    click(clear_field.webform_x_coord, clear_field.webform_y_coord)
    time.sleep(5)

    for field in nomogram_fields.values():
        field_name = field.web_field_name
        field_type = field.web_field_type
        if field_type == FieldType.CLEAR:
            continue
        webform_x_coord, webform_y_coord = field.webform_x_coord, field.webform_y_coord
        print(f"The field type of {field_name} is {field_type}")
        if field_type == FieldType.TEXTAREA:
            if field_name == "Current Age":
                value = "50"
            else:
                value = "1"
            doubleClick(webform_x_coord, webform_y_coord, interval=0.2)
            time.sleep(0.1)
            typewrite(value)
        elif field_type == FieldType.CHECKBOX:
            if field_name in ["Multifocality?", "Tumor is confined to UIQ?"]:
                value = "Yes"
            else:
                value = "No"
            if value == "Yes":
                click(webform_x_coord, webform_y_coord)
        elif field_type == FieldType.DROPDOWN:
            if field_name == "Tumor Type and Grade":
                value = "Ductal, II"
            elif field_name == "Estrogen-Receptor Status":
                value = "Negative"
            else:
                value = "Positive"
            click(webform_x_coord, webform_y_coord)
            time.sleep(0.1)
            option_x_coord, option_y_coord = field.get_dropdown_option_coords(value)
            time.sleep(0.1)
            click(option_x_coord, option_y_coord)
        elif field_type == FieldType.CALCULATE:
            click(webform_x_coord, webform_y_coord)
            time.sleep(1)
        elif field_type == FieldType.RESULT:
            time.sleep(1)
            top_left_coords = field.result_coords["top_left"]
            bottom_right_coords = field.result_coords["bottom_right"]
            moveTo(top_left_coords.x, top_left_coords.y)
            time.sleep(0.1)
            dragTo(bottom_right_coords.x, bottom_right_coords.y, duration=0.2, button="left")
            if os_type == "Windows":
                hotkey("ctrl", "c")
            elif os_type == "Mac":
                hotkey("command", "c")
            time.sleep(0.5)
            result = pyperclip.paste()
            if "%" in result:
                result = result.replace("%", "")
                result = round(float(result) / 100, 3)
                pyperclip.copy(result)
            print(f"The resulting probability is {result}")
            break
        else:
            raise Exception(f"Unknown field type: {field_type}")
        # Press tab to clear auto-completes
        time.sleep(0.1)
        press("tab")
    time.sleep(5)
    
# test_fill(nomogram_fields)

In [8]:
# Clear webform

def test_fill2(nomogram_fields):
    global os_type
    for field in nomogram_fields.values():
        field_name = field.web_field_name
        field_type = field.web_field_type
        if field_type == FieldType.CLEAR:
            value = None
        print(f"The field type of {field_name} is {field_type}")
        if field_type == FieldType.TEXTAREA:
            if field_name == "Current Age":
                value = "50"
            else:
                value = "1"
        elif field_type == FieldType.CHECKBOX:
            if field_name in ["Multifocality?", "Tumor is confined to UIQ?"]:
                value = "No"
            else:
                value = "Yes"
        elif field_type == FieldType.DROPDOWN:
            if field_name == "Tumor Type and Grade":
                value = "Lobular"
            elif field_name == "Estrogen-Receptor Status":
                value = "Negative"
            else:
                value = "Unknown"
        elif field_type == FieldType.CLEAR:
            value = None
        elif field_type == FieldType.CALCULATE:
            value = None
        elif field_type == FieldType.RESULT:    
            value = None
        else:
            raise Exception(f"Unknown field type: {field_type}")
        # Press tab to clear auto-completes
        
        field.fill_value(value)
    time.sleep(5)
    
test_fill2(nomogram_fields)

The field type of Clear is CLEAR
The field type of Current Age is TEXTAREA
The field type of Breast Tumor Size is TEXTAREA
The field type of Special Type? is CHECKBOX
The field type of Tumor is confined to UIQ? is CHECKBOX
The field type of Lymphatic or Vascular Structure Involvement (LymphovascularInvasion) is CHECKBOX
The field type of Multifocality? is CHECKBOX
The field type of Tumor Type and Grade is DROPDOWN
The field type of Estrogen-Receptor Status is DROPDOWN
The field type of Progesterone-Receptor Status is DROPDOWN
The field type of Calculate is CALCULATE
The field type of Probability of Spread to Sentinel Lymph Nodes (the area where the probability as a percentage will appear. Initially, it is an empty grey box, inside another grey box titled 'Your Results' and to the right to the blue text 'Probability of Spread to Sentinel Lymph Nodes'.) is RESULT
The resulting probability is 0.24


In [7]:
# Clear webform

def fill_patient_info(row, nomogram_fields):
    global os_type

    clear_field = nomogram_fields["Clear"]
    click(clear_field.webform_x_coord, clear_field.webform_y_coord)
    time.sleep(0.5)
    click(clear_field.webform_x_coord, clear_field.webform_y_coord)
    time.sleep(5)

    for field in nomogram_fields.values():
        field_name = field.web_field_name
        field_type = field.web_field_type
        if field_type == FieldType.CLEAR:
            continue
        webform_x_coord, webform_y_coord = field.webform_x_coord, field.webform_y_coord
        print(f"The field type of {field_name} is {field_type}")
        value = get_patient_value(row, field_name)
        field.fill_value(value)